In [2]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
import openai

/opt/homebrew/anaconda3/envs/trans_1225/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def read_guide(company, path):
    with open(path, 'r', encoding='utf-8') as file:
        guides = file.read().split("\n\n^^\n\n")
    
    last_guide = [f'[{company}] ' + guide for guide in guides]
    
    return last_guide

In [4]:
kakao_1 = read_guide('카톡', "./guide_file/katalk(1).txt")
kakao_1
./guide_file/katalk(1).txt

['[카톡] 오픈채팅 로컬탭 사용하기: 우리 지역 정보를 공유하고 이웃들과 관심사로 대화할 수 있습니다. 지금 같은 지역에 있는 사람들이 많이 찾는 장소는 어디일지, 이 동네의 관심사는 무엇인지 궁금하다면? 이제 오픈채팅 로컬탭에서 지역 정보도 공유하고 이웃들과 함께 관심사로 소통해 보세요. 1. 카카오톡 세 번째 오픈채팅탭을 클릭해 보세요. 2. 상단에서 우리 지역 이름을 클릭하세요. 3. 지금 사람들이 많이 찾는 장소의 순위와 정보를 확인할 수 있습니다. 4. 우리 지역 이웃들이 모여있는 오픈채팅 Lite에서 이야기를 나눌 수 있습니다. 5. 지금 우리 지역 이웃들의 관심사가 더 궁금하다면 추천된 다양한 주제의 오픈채팅방을 둘러보세요. * 사용자의 현 위치에 따라 로컬탭이 보입니다. * 오픈채팅 Lite는 채팅방에 들어가지 않고도 대화를 볼 수 있습니다.',
 "[카톡] 채팅방에서 송금하기: 메시지 보내듯 쉽고, 안전하게 송금하기를 이용해 보세요. 상대방의 계좌번호가 없어도 카카오톡 친구라면 간편하고 안전하게 송금을 할 수 있어요. 고마워요, 축하해요 등 함께 전하고 싶은 마음을 봉투에 담아 센스 있게 송금할 수도 있답니다. 1. 송금하려는 사람의 채팅창 하단의 + 버튼 > '송금'을 누릅니다. 2. '송금하기' 버튼을 선택합니다. 3. 보내고 싶은 금액을 입력합니다. 4. '봉투에 담아 보내기'를 클릭하면, 송금 상황에 맞는 다양한 봉투를 선택해서 송금할 수 있습니다. 5. 봉투를 선택했다면 봉투를 선택한 후, 아니라면 3번에서 바로 이어져서 '보내기' 버튼을 클릭합니다. 6. 안전하게 송금이 완료됩니다. 7. 송금봉투를 받았다면 받은 사람이 '송금 받기' 버튼을 눌러서 받을 수 있습니다. 8. 받은 사람의 카카오페이머니로 받기가 완료됩니다.",
 "[카톡] 다양한 방법으로 친구 추가하기: QR코드부터 카톡 ID까지, 다양한 방식으로 친구를 추가하고 간편하게 관리할 수 있습니다. 새로운 관계를 맺을 때마다 필요한 카카오톡 친구 추가. 꼭 전화번호를 주고받

In [3]:
kakao_1 = read_guide('카톡', "./guide_file/katalk(1).txt")
kakao_2 = read_guide('카톡', "./guide_file/katalk(2).txt")
baemin = read_guide('배민', "./guide_file/baemin.txt")
guppi = read_guide('구삐', "./guide_file/guppi.txt")

# 전체 가이드
guide_li = kakao_1 + kakao_2 + baemin + guppi

In [4]:
# 실질적으로 탑케이 필요함
embedder = SentenceTransformer("BM-K/KoSimCSE-roberta-multitask")
corpus_embeddings = embedder.encode(guide_li, convert_to_tensor=True)

No sentence-transformers model found with name /Users/parkjiyoung/.cache/torch/sentence_transformers/BM-K_KoSimCSE-roberta-multitask. Creating a new one with MEAN pooling.


In [5]:
# Query sentences:
query = input("질문을 입력하세요: ")

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
cnt = 1

# prompt list
prompt = []

# query 를 똑같이 임베딩 하여 가이드 뭉치와 코사인 유사도 비교
query_embedding = embedder.encode(query, convert_to_tensor=True)
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
cos_scores = cos_scores.cpu()

In [16]:
cos_scores

tensor([0.5611, 0.5625, 0.6258, 0.5863, 0.5775, 0.5865, 0.4529, 0.6004, 0.5414,
        0.5936, 0.5792, 0.5865, 0.5401, 0.5676, 0.5716, 0.5175, 0.5441, 0.5927,
        0.4692, 0.5289, 0.6218, 0.5108, 0.3864, 0.5237, 0.5085, 0.5647, 0.5891,
        0.5939, 0.5941, 0.5816, 0.5398, 0.4758, 0.6445, 0.5455, 0.5331, 0.5705,
        0.4621, 0.3880, 0.5431, 0.6341, 0.6546, 0.5571, 0.6344, 0.5119, 0.4867,
        0.5174, 0.4986, 0.4312, 0.6020, 0.4588, 0.3601, 0.6128, 0.6067, 0.5246,
        0.6120, 0.5408, 0.5310, 0.4164, 0.4452, 0.4322, 0.5628, 0.6014, 0.4472,
        0.6156, 0.5251, 0.4730, 0.5239, 0.5154, 0.5705, 0.5387, 0.5829, 0.5549,
        0.5657, 0.4246, 0.5668, 0.4335, 0.3527, 0.5718, 0.3089, 0.2762, 0.6316,
        0.5810, 0.6369, 0.6155, 0.6078, 0.5723, 0.5891, 0.4218, 0.5834, 0.6393,
        0.4373, 0.3716, 0.2628, 0.6611, 0.5098, 0.4784, 0.7228, 0.6731, 0.5704,
        0.6715, 0.5989, 0.6316, 0.4979, 0.4834, 0.5943, 0.5686, 0.5764, 0.5221,
        0.5045, 0.5001, 0.5487, 0.5864, 

In [15]:
np.argpartition(-cos_scores, range(top_k))

tensor([ 96,  97,  99,  93,  40,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,   4,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,   3,  94,  95,   0,   1,
         98,   2, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147])

In [8]:
#We use np.argpartition, to only partially sort the top_k results
top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
idxs = [x.item() for x in top_results]

In [9]:
top_results

tensor([96, 97, 99, 93, 40])

In [10]:
idxs

[96, 97, 99, 93, 40]

In [20]:
cos_scores

tensor([0.5611, 0.5625, 0.6258, 0.5863, 0.5775, 0.5865, 0.4529, 0.6004, 0.5414,
        0.5936, 0.5792, 0.5865, 0.5401, 0.5676, 0.5716, 0.5175, 0.5441, 0.5927,
        0.4692, 0.5289, 0.6218, 0.5108, 0.3864, 0.5237, 0.5085, 0.5647, 0.5891,
        0.5939, 0.5941, 0.5816, 0.5398, 0.4758, 0.6445, 0.5455, 0.5331, 0.5705,
        0.4621, 0.3880, 0.5431, 0.6341, 0.6546, 0.5571, 0.6344, 0.5119, 0.4867,
        0.5174, 0.4986, 0.4312, 0.6020, 0.4588, 0.3601, 0.6128, 0.6067, 0.5246,
        0.6120, 0.5408, 0.5310, 0.4164, 0.4452, 0.4322, 0.5628, 0.6014, 0.4472,
        0.6156, 0.5251, 0.4730, 0.5239, 0.5154, 0.5705, 0.5387, 0.5829, 0.5549,
        0.5657, 0.4246, 0.5668, 0.4335, 0.3527, 0.5718, 0.3089, 0.2762, 0.6316,
        0.5810, 0.6369, 0.6155, 0.6078, 0.5723, 0.5891, 0.4218, 0.5834, 0.6393,
        0.4373, 0.3716, 0.2628, 0.6611, 0.5098, 0.4784, 0.7228, 0.6731, 0.5704,
        0.6715, 0.5989, 0.6316, 0.4979, 0.4834, 0.5943, 0.5686, 0.5764, 0.5221,
        0.5045, 0.5001, 0.5487, 0.5864, 

In [21]:
guide_li[idxs[0]]

'[배민] 배달앱 설치하기: 1. 구글 플레이 스토어를 연다. 2. 검색창을 찾는다. 3. 배달의 민족을 입력하고 돋보기를 누른다. 4. 설치를 누른다. 5. 설치가 끝나면 배달의 민족을 사용할 수 있다.'

In [26]:
score = float("%.2f" % cos_scores[idxs[0]])
[score]

[0.72]

In [22]:




prompt.append("아래 정보들을 참고해서 질문에 대한 답을 알려줘. 정보 부분의 숫자는 질문과 얼마나 유사한지를 나타내는 지표고, 대괄호 안의 단어는 어플 이름이야. \
꼭 질문과 관련 없는 어플 이름은 사용하지 말고, 질문에 대해 필요한 정보만 사용해. 질문과 관련이 있는 질문에 대한 답변만 알려 주고, 질문에 대한 적절한 답이 없다면 답변이 불가하다고 답해.")
prompt.append(f"### 질문 ###")
prompt.append(query)

scores = []
for idx in idxs:
    if 0 <= idx < len(guide_li):
        score = float("%.2f" % cos_scores[idx])
        scores.append(score)
        if scores[0] >= 0.5:
            prompt.append(f"### 정보{cnt} ###")
            prompt.append("%.2f" % cos_scores[idx])

            prompt.append(guide_li[idx].strip())
        else:
            prompt.append("적절한 답이 없습니다.")
            break
    else:
        prompt.append("Index")
        prompt.append(idx)
        prompt.append("is out of range")
    cnt += 1

scores = []
cnt = 1

prompt = '\n'.join(prompt)
print(prompt)

아래 정보들을 참고해서 질문에 대한 답을 알려줘. 정보 부분의 숫자는 질문과 얼마나 유사한지를 나타내는 지표고, 대괄호 안의 단어는 어플 이름이야. 꼭 질문과 관련 없는 어플 이름은 사용하지 말고, 질문에 대해 필요한 정보만 사용해. 질문과 관련이 있는 질문에 대한 답변만 알려 주고, 질문에 대한 적절한 답이 없다면 답변이 불가하다고 답해.
### 질문 ###
배민 어플 사용 방법 알려줘
### 정보1 ###
0.72
[배민] 배달앱 설치하기: 1. 구글 플레이 스토어를 연다. 2. 검색창을 찾는다. 3. 배달의 민족을 입력하고 돋보기를 누른다. 4. 설치를 누른다. 5. 설치가 끝나면 배달의 민족을 사용할 수 있다.
### 정보2 ###
0.67
[배민] 주소 설정하기: 1. 배달의 민족 앱을 연다. 2. 동의하는 내용에 체크하고 시작하기를 누른다. ('필수'에는 꼭 체크해야 앱을 이용할 수 있다. '선택'에는 원하면 체크한다.). 3. 배달 받을 주소를 검색한다.(예. <서울시 중구 소소로123,101호>면, 소소로123만 입력한다.) 4. 상세 주소를 입력하고 완료를 누른다.(상세주소: 건물 이름이나 동, 호 등 주소의 뒷부분) 5. 배달 받을 주소가 설정되었다. 음식을 주문하면 내가 정한 주소로 배달 받을 수 있다.
### 정보3 ###
0.67
[배민] 첫 화면 이해하기: 1. 상단의 네모 네 개 = 서비스 전체 보기(배달의민족 앱에서 이용할 수 있는 모든 서비스를 볼 수 있다.) 2. 상단의 돋보기 = 검색(원하는 메뉴나 가게를 검색해 바로 찾을 수 있다.) 3. 배민1 = 빠른 배달(오른쪽의 배달보다 빠르게 받을 수 있다. 지역에 따라 '배민1과 '배달'의 위치가 다르며 '배민1'이 없을 수 있다.) 4. 포장 = 포장 주문하기(직접 가지러 가서 배달팁을 아낄 수 있다.) 6. 중앙 상단의 주소 = 주소 정하기(배달 받을 곳을 주문할 때 정할 수 있다. 주소를 우리집, 회사 등의 이름으로 정하면 편하다.) 7. 오른쪽 상단의 종 모양 = 알림센터(

In [23]:
prompt

"아래 정보들을 참고해서 질문에 대한 답을 알려줘. 정보 부분의 숫자는 질문과 얼마나 유사한지를 나타내는 지표고, 대괄호 안의 단어는 어플 이름이야. 꼭 질문과 관련 없는 어플 이름은 사용하지 말고, 질문에 대해 필요한 정보만 사용해. 질문과 관련이 있는 질문에 대한 답변만 알려 주고, 질문에 대한 적절한 답이 없다면 답변이 불가하다고 답해.\n### 질문 ###\n배민 어플 사용 방법 알려줘\n### 정보1 ###\n0.72\n[배민] 배달앱 설치하기: 1. 구글 플레이 스토어를 연다. 2. 검색창을 찾는다. 3. 배달의 민족을 입력하고 돋보기를 누른다. 4. 설치를 누른다. 5. 설치가 끝나면 배달의 민족을 사용할 수 있다.\n### 정보2 ###\n0.67\n[배민] 주소 설정하기: 1. 배달의 민족 앱을 연다. 2. 동의하는 내용에 체크하고 시작하기를 누른다. ('필수'에는 꼭 체크해야 앱을 이용할 수 있다. '선택'에는 원하면 체크한다.). 3. 배달 받을 주소를 검색한다.(예. <서울시 중구 소소로123,101호>면, 소소로123만 입력한다.) 4. 상세 주소를 입력하고 완료를 누른다.(상세주소: 건물 이름이나 동, 호 등 주소의 뒷부분) 5. 배달 받을 주소가 설정되었다. 음식을 주문하면 내가 정한 주소로 배달 받을 수 있다.\n### 정보3 ###\n0.67\n[배민] 첫 화면 이해하기: 1. 상단의 네모 네 개 = 서비스 전체 보기(배달의민족 앱에서 이용할 수 있는 모든 서비스를 볼 수 있다.) 2. 상단의 돋보기 = 검색(원하는 메뉴나 가게를 검색해 바로 찾을 수 있다.) 3. 배민1 = 빠른 배달(오른쪽의 배달보다 빠르게 받을 수 있다. 지역에 따라 '배민1과 '배달'의 위치가 다르며 '배민1'이 없을 수 있다.) 4. 포장 = 포장 주문하기(직접 가지러 가서 배달팁을 아낄 수 있다.) 6. 중앙 상단의 주소 = 주소 정하기(배달 받을 곳을 주문할 때 정할 수 있다. 주소를 우리집, 회사 등의 이름으로 정하면 편하다.) 7. 오른쪽 상단의 

In [14]:
with open("../pass_key/gpt_api.txt", "r") as key:
    pass_key = key.read()

openai.api_key = pass_key

def get_completion(prompt, model="gpt-4"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

response = get_completion(prompt)
print(response)

네, 배민에서 카카오페이로 결제가 가능합니다. 결제하기 중 다른 결제수단에 있는 '변경'을 누르면 '카카오페이'를 선택하여 결제를 진행할 수 있습니다.
